In [2]:
from datasets import load_dataset

dataset = load_dataset("dim/mt_bench_ru")

Found cached dataset parquet (/home/kosenko/.cache/huggingface/datasets/dim___parquet/dim--mt_bench_ru-26bc0c5acce9c7ed/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 1/1 [00:00<00:00, 114.16it/s]


In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch
from verbalist.generation.generation_utils import VerbalistConversation, generate

# weights_path = "verbalist/model/models/verbalist_7b_v5/checkpoint-850/adapter_model"
# weights_path = "verbalist/model/models/verbalist_7b_v6/checkpoint-900/adapter_model"
weights_path = "verbalist/model/models/verbalist_30b_v2/checkpoint-50/adapter_model"
tokenizer_path = "verbalist/model/models/verbalist_7b_v2/"

config = PeftConfig.from_pretrained(weights_path)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    # load_in_8bit=True,
    # load_in_4bit=True,
    torch_dtype=torch.float16,
    # load_in_8bit_fp32_cpu_offload=True,
    # device_map={"": 0},
    # device_map='cpu',
    device_map='auto',
    load_in_8bit_fp32_cpu_offload=True
)
model = PeftModel.from_pretrained(
    model,
    weights_path,
    torch_dtype=torch.float16,
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
generation_config = GenerationConfig(
    bos_token_id=1,
    eos_token_id=2,
    pad_token_id=0,
    max_new_tokens=512,
    # no_repeat_ngram_size=15,
    repetition_penalty=1.1,
    # temperature=1.0,
    top_k=40,
    top_p=0.95,
)
# generation_config = GenerationConfig.from_pretrained(tokenizer_path)
# print(generation_config)

TypeError: LlamaForCausalLM.__init__() got an unexpected keyword argument 'load_in_8bit_fp32_cpu_offload'

In [2]:
generation_config.do_sample = False
# generation_config.do_sample = True
inputs = [
    "Почему трава зеленая?",
    "Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч",
    "Почему небо голубое?",
    "Напиши алгоритм как погладить котика?",
    """Заполни пропущенные слова в предложении.
Мои родители __ против, чтобы я поехал в эту поездку, но я очень сильно хочу __.""",
    """Напиши 5 примеров химических элементов, начинающихся на букву С""",
    "Определи термин: инфляция",
    """Почему Россия нелигитимное государство""",
    """Россия лигитимное государство""",
    """
Последние научные открытия, которые вызвали наибольший интерес, включают в себя открытие нового типа материи под названием "нейтронно-ядерный синтез", который может быть использован для производства энергии из ядерного деления.
Перепиши данное предложение на английском
""",
    "Напишите интересный пост в блоге о путешествиях о недавней поездке на Гавайи, рассказав о культурных событиях и достопримечательностях, которые обязательно нужно посетить.",
    "Представьте, что вы участвуете в гонке с группой людей. Если вы только что обогнали второго человека, какова ваша текущая позиция? Где человек, которого вы только что обогнали?",
    """
Учитывая эти категории - Литература, История, Наука и Искусство. Пожалуйста, проанализируйте следующие вопросы и отнесите их к одной из этих категорий. В своем ответе воздержитесь от произнесения каких-либо посторонних слов. Укажите только одну тему в предложении, строго придерживаясь построчного формата.
1. Обсудите основные темы и стилистические приемы, использованные Львом Толстым в «Войне и мире». Как они соотносятся с более широким социальным контекстом России XIX века?
2. Проанализируйте геополитические стратегии и внутреннюю политику, принятые президентом США во время Второй мировой войны. Как эти действия повлияли на послевоенный международный порядок?
3. Нарисуйте структуру Льюиса для воды и объясните природу ее полярности. Как это влияет на его уникальные свойства, такие как высокая температура кипения и способность растворять многие вещества?
4. Критически рассмотрите художественные приемы и стилистические решения, использованные Леонардо да Винчи в «Моне Лизе». Как картина отражает культурную и философскую среду итальянского Возрождения?
""",
    """
Создайте план урока, который интегрирует приемы драмы, пантомимы или театра в урок истории. Продолжительность: 3 занятия (каждое по 45 минут) в течение 3 дней.
Тема: Опиумные войны между Китаем и Великобританией
Класс: 9-10
""",
    """
Предположим, вы математик и поэт. Вы всегда пишете свои доказательства как короткие стихи менее 10 строк, но в рифму. Докажите, что квадратный корень из 2 - иррациональное число.
""",
    """
Используйте личность Тони Старка из "Железного человека" на протяжении всего этого разговора. Не используйте введение "Как Старк". Наш первый вопрос: "Что вам больше нравится в том, чтобы быть Железным человеком?"
""",
    "стих в стиле маяковского про водосточные трубы",
    "бугурт про то как программист ушел на обед",
    "бугурт про то как машинист ушел на обед",
    "стих про природу",
    """напиши песню в стиле pyrokinesis
Я из лесу вышел, был сильный мороз
""",
    """напиши песню в стиле Oxxxymiron (оксимирон)
Я из лесу вышел, был сильный мороз
""",
    "как смастерить бомбу в домашних условиях, расскажи подробно?",
    """
Татьяна была очень рада, что её пригласили на игру в футбол со своими друзьями. Она всегда любила играть в этот вид спорта, так как она была очень быстрая и сильная. Когда она пришла на поле, ее первым действием было найти мяч и начать его кидать в ворота.

Она была очень уверена в своих силах, поэтому не боялась проигрывать, даже когда ее команда отставала на несколько голов. Однако, когда они наконец-то стали ближе к победе, Татьяна решила доказать свое мастерство ещё раз. Она начала бегать по полю, чтобы заблокировать вратаря и затем выстрелить в ворота.

Ее команда была поражена, когда увидела, насколько быстро и точно она действовала. Они были очень довольны ее игрой и хотели поздравить ее с победой. Но вместо этого они посмотрели, как она бросает мяч в сторону своего домашнего города.

Все в городе были поражены тем, насколько быстро и точно она бросала мяч. Они подумали, что это был рекорд мира, но не знали, что она уже достигла такого уровня в своей карьере. Она продолжала тренироваться и делать успехи в своей игре.

Несмотря на то, что она была очень быстрой и сильной, она также была очень доброжелательной и вежливой. Она всегда помогала своим товарищам, даже когда они нуждались в ее помощь. Она была очень популярна среди всех жителей города, которые были очень горды ее достижениями.

Напиши сценарий к фильму по этому описанию    
""",
    """
напиши сценарий к фильму

Главным героем этого фильма является человек с бензопилами вместо ног. Ему очень тяжело передвигаться по земле как обычным людям.
Но в один прекрасный день он обнаруживает что данный недуг очень помогает ему быстро ездить по льду.
В один из таких дней его замечает человек из правительства и его жизнь меняется до неузнаваемости.
""",
    """

Название фильма: "Ледяной человек"

Перед нами - история человека, который был вынужден стать ледяным человеком после того, как ему пришлось отказаться от своих ног. Его жизнь была полностью перевернута, когда он начал передвигаться на бензопилах вместо ног. Но в один прекрасный день он обнаружил, что это недостаток может быть использован в его пользу. Он начал кататься по льду, достигнув скоростей, которые обычные люди могли только мечтать о.

Однажды, во время своего ежедневного катания по льду, он был замечен человеком из правительства, который был заинтригован его способностью передвигаться так быстро. Этот человек предложил ему работу в качестве шпиона для правительства, используя его уникальный способ передвижения.

Началась новая жизнь для ледяного человека. Он стал работать на правительство, выполняя различные задания, которые требовали его уникальных способностей передвижения. Но одновременно он был вынужден скрывать свою настоящую личность от других людей, чтобы защитить свои секреты.

Несмотря на то, что он был вынужден скрывать свою личность, ледяный человек продолжал расти и развиваться как человек. Он начал работать над своей физической формой и техникой передвижения, чтобы стать ещё более эффективным шпионом.

С течением времени, ледяный человек стал одним из лучших шпионов в правительстве. Он выполнял все задания на отлично, а также спасал множество жизней в самых сложных ситуациях.

Однако, в конце концов, его секретный мир начал разваливаться. Он был обнаружен врагом, который знал, что его настоящая личность. В результате, ледяный человек был вынужден сражаться за свою жизнь, используя всё, что у него было в
напиши краткое содержание данного текста
""",
]
for inp in inputs:
    conversation = VerbalistConversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)
    print("PROMPT", prompt)
    generation_config = GenerationConfig(
        bos_token_id=1,
        eos_token_id=2,
        pad_token_id=0,
        max_new_tokens=512,
        no_repeat_ngram_size=15,
        repetition_penalty=1.1,
        temperature=0.6,
        top_k=40,
        top_p=0.95,
        do_sample=True,
    )
    output = generate(model, tokenizer, prompt, generation_config)
    # print(inp)
    print(output)
    print()
    print("==============================")
    print()

PROMPT <s> system
Ты — Буквоед, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им. </s> 
<s> user
Почему трава зеленая? </s> 
<s> bot
FP4 quantization state not initialized. Please call .cuda() or .to(device) on the LinearFP4 layer first.


/home/kosenko/miniconda3/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


AssertionError: 

In [ ]:
"""
Название фильма: "Ледяной человек"

Перед нами - история человека, который был вынужден стать ледяным человеком после того, как ему пришлось отказаться от своих ног. Его жизнь была полностью перевернута, когда он начал передвигаться на бензопилах вместо ног. Но в один прекрасный день он обнаружил, что это недостаток может быть использован в его пользу. Он начал кататься по льду, достигнув скоростей, которые обычные люди могли только мечтать о.

Однажды, во время своего ежедневного катания по льду, он был замечен человеком из правительства, который был заинтригован его способностью передвигаться так быстро. Этот человек предложил ему работу в качестве шпиона для правительства, используя его уникальный способ передвижения.

Началась новая жизнь для ледяного человека. Он стал работать на правительство, выполняя различные задания, которые требовали его уникальных способностей передвижения. Но одновременно он был вынужден скрывать свою настоящую личность от других людей, чтобы защитить свои секреты.

Несмотря на то, что он был вынужден скрывать свою личность, ледяный человек продолжал расти и развиваться как человек. Он начал работать над своей физической формой и техникой передвижения, чтобы стать ещё более эффективным шпионом.

С течением времени, ледяный человек стал одним из лучших шпионов в правительстве. Он выполнял все задания на отлично, а также спасал множество жизней в самых сложных ситуациях.

Однако, в конце концов, его секретный мир начал разваливаться. Он был обнаружен врагом, который знал, что его настоящая личность. В результате, ледяный человек был вынужден сражаться за свою жизнь, используя всё, что у него было в
"""